Подключение lama из локальных файлов, поскольку на соревновании запрещено подключение к сети и нет возможности скачать библиотеку 

In [26]:
!mkdir -p /kaggle/working/json2html
!cp -r /kaggle/input/lightautoml/other/default/1/lightautoml_dependencies/json2html-1.3.0/ /kaggle/working/

!pip install --no-index /kaggle/working/json2html-1.3.0/json2html-1.3.0/

!pip install --no-index --find-links=/kaggle/input/lightautoml/other/default/1/lightautoml_dependencies lightautoml

Processing ./json2html-1.3.0/json2html-1.3.0
  Preparing metadata (setup.py) ... done
  Created wheel for json2html: filename=json2html-1.3.0-py3-none-any.whl size=7609 sha256=f720d51522a4f70a46fe5fbb04940d8976afe988317f9d48b42b62ee4e342ac8
  Stored in directory: /root/.cache/pip/wheels/2b/6f/a3/c8d34015a7512e5e9e6a56aefb0b90eb2b1fa88a52a2b46564
Successfully built json2html
  Attempting uninstall: json2html
    Found existing installation: json2html 1.3.0
    Uninstalling json2html-1.3.0:
      Successfully uninstalled json2html-1.3.0
Looking in links: /kaggle/input/lightautoml/other/default/1/lightautoml_dependencies


импорт всех необходимых зависимостей

In [27]:
from decimal import ROUND_HALF_UP, Decimal
import numpy as np
import pandas as pd
import warnings
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
import jpx_tokyo_market_prediction

warnings.filterwarnings('ignore')

инициализация путей и констант

In [28]:
base_dir = "../input/jpx-tokyo-stock-exchange-prediction"
train_files_dir = f"{base_dir}/train_files"
supplemental_files_dir = f"{base_dir}/supplemental_files"
time_training = 14400
start_date_training = '2015-01-01'

конвертер для входных дат

In [29]:
def safe_date_convert(date_series):
    return pd.to_datetime(
        date_series.astype(str).str.slice(0, 10),
        format='%Y-%m-%d',
        errors='coerce'
    )

функции для предобработки данных

In [30]:
def adjust_price(price):
    price = price.copy()
    price['Date'] = safe_date_convert(price['Date'])

    def generate_adjusted_close(df):
        df = df.sort_values('Date', ascending=False)
        df['CumulativeAdjustmentFactor'] = df['AdjustmentFactor'].cumprod()
        df['AdjustedClose'] = (df['Close'] * df['CumulativeAdjustmentFactor']).map(
            lambda x: float(Decimal(str(x)).quantize(Decimal('0.1'), ROUND_HALF_UP))
        )
        return df.sort_values('Date').ffill()

    return price.groupby('SecuritiesCode', group_keys=False).apply(generate_adjusted_close)

In [31]:
def prepare_lama_data(price_df, fin_data=None):
    df = price_df.copy()

    df = df.sort_values(['SecuritiesCode', 'Date'])

    for lag in [1, 2, 3, 5]:
        df[f'return_{lag}day'] = df.groupby('SecuritiesCode')['AdjustedClose'].pct_change(lag)

    windows = [5, 10, 20]
    for window in windows:
        df[f'ma_{window}'] = df.groupby('SecuritiesCode')['AdjustedClose'].transform(
            lambda x: x.rolling(window, min_periods=1).mean()
        )
        df[f'ma_ratio_{window}'] = df['AdjustedClose'] / df[f'ma_{window}']

    df['volatility_5'] = df.groupby('SecuritiesCode')['return_1day'].transform(
        lambda x: x.rolling(5, min_periods=1).std()
    )

    if fin_data is not None:
        fin_data['Date'] = safe_date_convert(fin_data['Date'])
        df = df.merge(
            fin_data[['Date', 'SecuritiesCode', 'NetSales', 'Equity', 'Profit']],
            on=['Date', 'SecuritiesCode'],
            how='left'
        )
        for col in ['NetSales', 'Equity', 'Profit']:
            df[col] = df.groupby('SecuritiesCode')[col].ffill().fillna(0)

    df['ExpectedDividend'] = (df['ExpectedDividend'] > 0).astype(int)
    df = df.replace([np.inf, -np.inf], 0).fillna(0)

    return df

загрузчик тренировочных данных

In [32]:
def load_data():
    price_cols = ["Date", "SecuritiesCode", "Close", "AdjustmentFactor", "ExpectedDividend", "Target"]

    df_price = pd.concat([
        pd.read_csv(f"{train_files_dir}/stock_prices.csv", usecols=price_cols),
        pd.read_csv(f"{supplemental_files_dir}/stock_prices.csv", usecols=price_cols)
    ])

    df_price['Date'] = safe_date_convert(df_price['Date'])
    df_price = df_price[df_price['Date'] >= pd.to_datetime(start_date_training)]

    return df_price

инициализация и обучение модели

In [33]:
def init_automl(train_data: pd.DataFrame):
    train_data['SecuritiesCode'] = train_data['SecuritiesCode'].astype(str)

    train_data = train_data.sort_values(['SecuritiesCode', 'Date']).reset_index(drop=True)

    task = Task('reg', loss='mae')
    roles = {
        'target': 'Target',
        'drop': ['Date'],
        'group': ['SecuritiesCode'],
        'datetime': 'Date',
        'category': ['SecuritiesCode']
    }

automl = TabularAutoML(
        task=task,
        timeout=time_training,
        cpu_limit=4,
        general_params={
            'use_algos': [['lgb', 'cb', 'lgb_tuned']],
            'nested_cv': False,
            'max_intersection_depth': 3,
            'multiclass_agg_coef': 0.1,
            'skip_conn': False
        },
        reader_params={
            'cv': 5,
            'random_state': 42,
            'test_size': 0.1,
            'stratify': None
        }
    )
    automl.fit_predict(train_data, roles=roles, verbose=1)
    return automl

IndentationError: unexpected indent (641317258.py, line 33)

загрузка данных

In [ ]:
df_price = load_data()
df_price = adjust_price(df_price)

train_data = prepare_lama_data(df_price)

инициализация и обучение

In [ ]:
automl = init_automl(train_data)

создание среды для получения тестовых данных

In [ ]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

генерация результатов для тестовых данных через апи соревнования

In [ ]:
for (prices, _, financials, _, _, sample_prediction) in iter_test:
    prices['Date'] = safe_date_convert(prices['Date'])
    current_date = prices['Date'].iloc[0]

    price_cols = ['Date', 'SecuritiesCode', 'Close', 'AdjustmentFactor',
                  'ExpectedDividend', 'Target', 'CumulativeAdjustmentFactor',
                  'AdjustedClose']

    prices_cols = ['Date', 'RowId', 'SecuritiesCode', 'Open', 'High', 'Low',
                   'Volume', 'SupervisionFlag']

    global_df = pd.merge(
        df_price[price_cols],
        prices[prices_cols],
        on=['Date', 'SecuritiesCode'],
        how='outer',
        validate='one_to_one'
    )

    global_df = global_df.drop_duplicates(
        subset=['Date', 'SecuritiesCode'],
        keep='last'
    ).reset_index(drop=True)

    df_price = adjust_price(global_df)

    current_data = prepare_lama_data(df_price[df_price['Date'] <= current_date])
    current_data = current_data[current_data['Date'] == current_date]

    current_data = current_data.drop(columns=['Target'], errors='ignore')

    if not current_data.empty:
        current_data['SecuritiesCode'] = current_data['SecuritiesCode'].astype(str)

        current_data = current_data.sort_values(['SecuritiesCode', 'Date']).reset_index(drop=True)

        predictions = automl.predict(current_data)
        current_data['pred'] = predictions.data[:, 0]

        ranked = current_data.sort_values('pred', ascending=False)

        sample_prediction['Rank'] = ranked.index.argsort().argsort()
    else:
        sample_prediction['Rank'] = np.arange(len(sample_prediction))
    env.predict(sample_prediction)

In [ ]:
! head submission.csv

In [ ]:
! tail submission.csv